In [1]:
import urdf2casadi.urdfparser as u2c
import numpy as np

## 1. Load robot from urdf 

1. Create urdfparser-class instance. 
2. Load model to instance, either from file, string or ros parameter server. Examples uses from file. 

In [2]:
ur5 = u2c.URDFparser()
ur5.from_file("/home/lillmaria/urdf2casadi/examples/urdf/ur5.urdf")

/home/lillmaria/urdf2casadi/examples/urdf/ur5.urdf


## 2. Get joint information ? 
Information about the joints of the robot model can be obtained using u2c's get_joint_info(). "root" and "tip" are the inputs, and represent the link names for the root and tip of the kinematic tree one wishes to evaluate. 

In [3]:
root = "base_link"
tip = "wrist_3_link"

joint_list, joint_names, q_max, q_min = ur5.get_joint_info(root, tip)
n_joints = len(joint_list)
print "name of first joint:", joint_names[0], "\n"
print "joint information for first joint:\n", joint_list[0]
print "\n q max:", q_max
print "\n q min:", q_min

ur5.get_forward_dynamics_ABA_2(root, tip)

name of first joint: shoulder_pan_joint 

joint information for first joint:
axis:
- 0.0
- 0.0
- 1.0
calibration: None
child: shoulder_link
dynamics:
  damping: 0.0
  friction: 0.0
limit:
  effort: 150.0
  lower: -6.28318530718
  upper: 6.28318530718
  velocity: 3.15
mimic: None
name: shoulder_pan_joint
origin:
  rpy:
  - 0.0
  - 0.0
  - 0.0
  xyz:
  - 0.0
  - 0.0
  - 0.089159
parent: base_link
safety_controller: None
type: revolute

 q max: [6.28318530718, 6.28318530718, 3.14159265359, 6.28318530718, 6.28318530718, 6.28318530718]

 q min: [-6.28318530718, -6.28318530718, -3.14159265359, -6.28318530718, -6.28318530718, -6.28318530718]


AttributeError: 'URDFparser' object has no attribute 'get_forward_dynamics_ABA_2'

# 3. Obtain robot dynamics! 
After loading the robot model to the urdfparser-instance, the robot dynamics can be obtained from any given root and tip of the robot. 

## Dynamic parameters:
To obtain the dynamic parameters (M, C, G) of the equation of motion tau = M(q)q_ddot + C(q, q_dot)q_dot + G(q):

In [4]:
M_sym = ur5.get_jointspace_inertia_matrix(root, tip)
C_sym = ur5.get_jointspace_bias_matrix(root, tip)

#gravity as defined on earth, 9.81m/s in negative z-axis:
gravity = [0, 0, -9.81]
G_sym = ur5.get_gravity_RNEA(root, tip, gravity)

M_sym, C_sym, G_sym are CasADi symbolic expressions of the ur5's dynamic parameters from given root to tip. The CasADi expressions are C compiled so they can be numerically evaluated efficiently. Example of numerical evaluation is shown below. 

In [5]:
q = [None]*n_joints
q_dot = [None]*n_joints
for i in range(n_joints):
    #to make sure the inputs are within the robot's limits:
    q[i] = (q_max[i] - q_min[i])*np.random.rand()-(q_max[i] - q_min[i])/2
    q_dot[i] = (q_max[i] - q_min[i])*np.random.rand()-(q_max[i] - q_min[i])/2
    
M_num = M_sym(q)
C_num = C_sym(q, q_dot)
G_num = G_sym(q)
print "Numerical Inertia Matrx: \n", M_num 
print "\nNumerical Coriolis term: \n", C_num 
print "\nNumerical gravity term: \n", G_num

Numerical Inertia Matrx: 

[[89.2118, -7.96692e-11, 5.82581, 1.30429, -0.0337024, -0.00808052], 
 [-7.96692e-11, 13.2939, -0.282569, 0, 0, 0], 
 [5.82581, -0.282569, 0.839714, -0, -0.0034105, -0.0150505], 
 [1.30429, 0, -0, 2.6259, 0, 0], 
 [-0.0337024, 0, -0.0034105, 0, 0.240572, 0], 
 [-0.00808052, 0, -0.0150505, 0, 0, 0.0171365]]

Numerical Coriolis term: 
[-8.48192, 6.82591, 2.49823, -39.1171, -0.63447, -0.158858]

Numerical gravity term: 
[-7.9585e-15, -130.413, 2.772, -8.88178e-16, 0, 0]


## Inverse Dynamics 

Without accounting for gravitational forces:

In [6]:
tau_sym = ur5.get_inverse_dynamics_RNEA(root, tip)

Accounting for gravitational forces:

In [7]:
gravity = [0, 0, -9.81]
tau_g_sym = ur5.get_inverse_dynamics_RNEA(root, tip, gravity = gravity)

External forces can also be accounted for:

In [41]:
tau_ext = np.ones(n_joints)
tau_fext_sym = ur5.get_inverse_dynamics_RNEA(root, tip, f_ext = tau_ext)

NameError: global name 'i_X_0' is not defined

In [8]:
q = [None]*n_joints
q_dot = [None]*n_joints
q_ddot = [None]*n_joints
for i in range(n_joints):
    #to make sure the inputs are within the robot's limits:
    q[i] = (q_max[i] - q_min[i])*np.random.rand()-(q_max[i] - q_min[i])/2
    q_dot[i] = (q_max[i] - q_min[i])*np.random.rand()-(q_max[i] - q_min[i])/2
    q_ddot[i] = (q_max[i] - q_min[i])*np.random.rand()-(q_max[i] - q_min[i])/2

tau_num = tau_sym(q, q_dot, q_ddot)
tau_g_num = tau_g_sym(q, q_dot, q_ddot)
#tau_fext_num = tau_fext_sym(q, q_dot, q_ddot)
print "Numerical inverse dynamics: \n", tau_num 
print "\nNumerical inverse dynamics w/ gravity: \n", tau_g_num 
print "\nNumerical inverse dynamics w/ external forces: \n", G_num

Numerical inverse dynamics: 
[-9.91911, 58.0452, -1.48518, -10.8123, 1.06382, 0.104966]

Numerical inverse dynamics w/ gravity: 
[-9.91911, -72.368, -9.7009, -10.8123, 1.06382, 0.104966]

Numerical inverse dynamics w/ external forces: 
[-7.9585e-15, -130.413, 2.772, -8.88178e-16, 0, 0]


## Forward Dynamics

urdf2casadi provides two methods for finding the robot's forward dynamics. The first method combines the Recursive Newton-Euler Algorithm (RNEA) and the Composite Rigid Body Algorithm (CRBA) and solves the equation of motion for the joint accelerations. The second method uses the Articulated Body Algorithm (ABA) for forward dynamics. The method that uses ABA is in most cases the most efficient with regard to numerical evaluation, especially if the number of joints are high. (See timing examples for more information.)

In [9]:
tau = np.zeros(n_joints)
qddot_sym = ur5.get_forward_dynamics_CRBA(root, tip, tau)

In [10]:
qddot_g_sym = ur5.get_forward_dynamics_ABA(root, tip, tau, gravity = gravity)
qddot_sym_2 = ur5.get_forward_dynamics_ABA_2(root, tip)

AttributeError: 'URDFparser' object has no attribute 'get_forward_dynamics_ABA_2'

In [ ]:
q = [None]*n_joints
q_dot = [None]*n_joints
for i in range(n_joints):
    #to make sure the inputs are within the robot's limits:
    q[i] = (q_max[i] - q_min[i])*np.random.rand()-(q_max[i] - q_min[i])/2
    q_dot[i] = (q_max[i] - q_min[i])*np.random.rand()-(q_max[i] - q_min[i])/2

qddot_num = qddot_sym(q, q_dot)
qddot_g_num = qddot_g_sym(q, q_dot)